In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



## download dataset

In [ ]:
def download_mnist_datasets():
    train_data = datasets.MNIST(
        root = "data" 
        download=True
        train=True
        transform=ToTensor() # each value is normalized btw 0 and 1
    )

    validation_data = datasets.MNIST(
        root = "data" 
        download=True
        train=False
        transform=ToTensor() # each value is normalized btw 0 and 1
    )
    return train_data, validation_data

In [ ]:
train_data, _ = downlaod_mnist_datasets()